In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/nprhealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585113044621066240,Mon Apr 06 16:13:15 +0000 2015,RT @JennyAGold: Maybe You Should Skip That Ann...
1,547824165185536000,Wed Dec 24 18:40:33 +0000 2014,Would You Like Health Insurance With Those Sto...
2,547763897638199296,Wed Dec 24 14:41:04 +0000 2014,New Blood Donation Rules Would Still Exclude M...
3,547671560471859200,Wed Dec 24 08:34:09 +0000 2014,Obama Administration Downplays Court Challenge...
4,547671553609981952,Wed Dec 24 08:34:08 +0000 2014,"Christmas In Liberia: Ebola Fears, No Snow, Ho..."


In [3]:
# -------------------------------------------------
frame.shape

(4838, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'RT @JennyAGold: Maybe You Should Skip That Annual Physical @nprnews @KHNews http://www.npr.org/blogs/health/2015/04/06/397100748/maybe-you-should-skip-that-annual-physical'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)
frame.head()

http://www.npr.org/blogs/health/2015/04/06/397100748/maybe-you-should-skip-that-annual-physical
http://n.pr/1CKulog
http://n.pr/1CJ9IsG
http://n.pr/1CHaDd6
http://n.pr/1CHaAxQ
http://n.pr/1CHaCFX
http://n.pr/1CwBBH8
http://n.pr/1CvRzBi
http://n.pr/1CC0eiQ
http://n.pr/1CtPkOZ
http://n.pr/1Cr8sND
http://n.pr/1Cr4a8N
http://n.pr/1CqGvp1
http://n.pr/1CsVIDk
http://n.pr/1Cix2QD
http://n.pr/1ChZQsy
http://n.pr/1CnbTlE
http://n.pr/1CjvIdt
http://n.pr/1ChSKS0
http://n.pr/1CevKmT
http://n.pr/1CdsRmf
http://n.pr/1C94qcz
http://n.pr/1C8UmjJ
http://n.pr/1Cc0BR7
http://n.pr/1CaudhG
http://n.pr/1AlhgSS
http://n.pr/1AkGJvH
http://n.pr/1AjMnOu
http://n.pr/1AjMmKt
http://n.pr/1C5v03t
http://n.pr/1AiXmIf
http://n.pr/1AiXmHW
http://n.pr/1C0mZwu
http://n.pr/1DPOyxc
http://n.pr/1DPJXLF
http://n.pr/1BZpjDZ
http://n.pr/1BYXPyz
http://n.pr/1BWBYaV
http://n.pr/1BT7A19
http://n.pr/1BT7CpE
http://n.pr/1DIpVme
http://n.pr/1DIlyrp
http://n.pr/1qXjRAz
http://n.pr/1BS7y9E
http://n.pr/1BRMi3O
http://n.pr/1BQvBpm
http

http://n.pr/1kT9rrL
http://n.pr/1kSPnWz
http://n.pr/1kIDlmZ
http://n.pr/1kQoAds
http://n.pr/1kQm4nk
http://n.pr/1kQgGk8
http://n.pr/1kNN5b1
http://n.pr/1kNxGaD
http://n.pr/1kCA4pm
http://n.pr/1nlECR4
http://n.pr/1nkDPQj
http://n.pr/1kAnbfu
http://n.pr/UcR38p
http://n.pr/1kyA0a5
http://n.pr/1kyumET
http://n.pr/1kyiSkQ
http://n.pr/1kxhJKq
http://n.pr/1kv0ycn
http://n.pr/1nazu21
http://n.pr/1kuGqqH
http://n.pr/1n9Czzv
http://n.pr/1n9CxaJ
http://n.pr/1ktZHIU
http://n.pr/1ktog8Z
http://n.pr/1n7MOnP
http://n.pr/1n5F31O
http://n.pr/1kqYuSP
http://n.pr/1n4ZGLv
http://n.pr/1kqnv09
http://n.pr/1kqgJrr
http://n.pr/1kq6IKG
http://n.pr/1kq6Iuc
http://n.pr/1korFG2
http://n.pr/1n2TVxY
http://n.pr/1mZxMR0
http://n.pr/1kh0ywo
http://n.pr/1kh0wVg
http://n.pr/1mUHrs8
http://n.pr/1mS6HPL
http://n.pr/1mS2BqJ
http://n.pr/1ke7udG
http://n.pr/1kdkwbs
http://n.pr/1mR58Si
http://n.pr/1mQGPUx
http://n.pr/1mQC90E
http://n.pr/1mQsWFR
http://n.pr/1kbjX1S
http://n.pr/1kaU962
http://n.pr/1mMQECO
http://n.pr/1mMgEy2
h

http://n.pr/J9OEVy
http://n.pr/J8M7eb
http://n.pr/JMHh7L
http://n.pr/J8dnJU
http://n.pr/JKtRca
http://n.pr/JKpoGt
http://n.pr/J6ufAw
http://n.pr/J6ifz2
http://n.pr/J6fn57
http://n.pr/JISWnR
http://n.pr/JICzrt
http://n.pr/JIq9Qs
http://n.pr/JIcXem
http://n.pr/1jilZhO
http://n.pr/JHWI0Y
http://n.pr/1kUZScC
http://n.pr/J4R1Jd
http://n.pr/J4tUyv
http://n.pr/JEOIO4
http://HealthCare.gov
http://n.pr/JEAzAs
http://n.pr/J3i33E
http://n.pr/JEvV5E
http://n.pr/JE8Xvg
http://n.pr/JE3N2D
http://n.pr/J2Nz1H
http://n.pr/J2KpuK
http://n.pr/J2p04X
http://nyti.ms/1hXMt6u
http://n.pr/JAp0KD
http://n.pr/J0Grmp
http://n.pr/J0DdiT
http://n.pr/J0upcN
http://n.pr/JA6yC2
http://n.pr/J0k14J
http://n.pr/YxCYxY
http://n.pr/JqXfna
http://n.pr/IZnkJx
http://n.pr/IZlwAa
http://n.pr/IZdbwo
http://n.pr/JvLDjk
http://n.pr/Jusw9t
http://n.pr/IWlRDN
http://n.pr/JokTRZ
http://n.pr/IUtvyp
http://n.pr/JnYOD2
http://n.pr/IUaBYv
http://n.pr/IU69ci
http://n.pr/ITR2zp
http://n.pr/ITORMi
http://n.pr/JmK9I9
http://n.pr/ITCBeV
htt

http://n.pr/Zyz89T
http://n.pr/11NX1KJ
http://n.pr/Zxafvg
http://n.pr/11MY4ue
http://n.pr/10PM0tl
http://n.pr/10MhPDi
http://n.pr/ZsqKZt
http://n.pr/10M3XZW
http://n.pr/11HtGVT
http://n.pr/ZSCjJd
http://n.pr/X7Byxe
http://n.pr/YuvgKi
http://n.pr/ZmrQG9
http://n.pr/ZOpXle
http://n.pr/10B1Fg6
http://n.pr/ZIoKff
http://n.pr/WW4h8k
http://n.pr/10w4MWC
http://n.pr/10tC7kZ
http://n.pr/11sRooX
http://n.pr/11q554b
http://n.pr/10sclh3
http://n.pr/YcOuE3
http://n.pr/11oTZQB
http://n.pr/ZBrnQ1
http://n.pr/ZAtRy8
http://n.pr/10jx0nn
http://n.pr/ZwHka8
http://n.pr/WICI2h
http://n.pr/15qCNXy
http://n.pr/ZuCbzy
http://n.pr/10em1eU
http://n.pr/YS1v2v
http://n.pr/YS1xXV
http://n.pr/15mhsyx
http://n.pr/XXUUqw
http://n.pr/XXUUqu
http://n.pr/15iTRyE
http://n.pr/116fxO9
http://n.pr/Wywou1
http://n.pr/ZnyoE6
http://n.pr/YJZMMm
http://n.pr/XGODPV
http://n.pr/157avkO
http://t.co/lW6UO7aC
http://ow.ly/ipo4v
http://n.pr/XFg5gE
http://n.pr/XFg50m
http://n.pr/155MBWI
http://n.pr/ZLCfJX
http://flip.it/6JHk8
http:/

http://n.pr/1wU2fCm
Native Americans Urged To Sign Up For Private Insurance http://n.pr/1zyUHbe


http://n.pr/1zyUHbe
Doctor Shortage Looming? Maybe Not http://n.pr/1xi5B5V


http://n.pr/1xi5B5V
Guarding The Ebola Border http://n.pr/1xi5yqW


http://n.pr/1xi5yqW
For Babies, Preterm Birth Is Now The Number One Cause of Death http://n.pr/1zxq6La


http://n.pr/1zxq6La
Toxic Tau of Alzheimer's May Offer A Path To Treatment http://n.pr/1zxq6uC


http://n.pr/1zxq6uC
Are Modafinil's Brain-Boosting Benefits Hype Or Science? http://n.pr/1qgcCUd


http://n.pr/1qgcCUd
Mourners Gather To Remember Surgeon Who Died Of Ebola http://n.pr/1qgcAM2


http://n.pr/1qgcAM2
Meditating On And Studying The Tau Of Alzheimer's http://n.pr/1qgcAvy


http://n.pr/1qgcAvy
The Guy Who Delivers HIV Medicine On His Bicycle http://n.pr/1qg5KpW


http://n.pr/1qg5KpW
In California, That MRI Will Cost You $255 — Or $6,221 http://n.pr/11lnUdD


http://n.pr/11lnUdD
The Story Of A 4-Year-Old Boy Helps Explain An Ebola Surge h

...

Back To Work After A Baby, But Without Health Insurance http://n.pr/1c7qYNX


http://n.pr/1c7qYNX
Small Businesses May Find Insurance Relief In Exchanges http://n.pr/171tzkL


http://n.pr/171tzkL
Studying The Science Behind Child Prodigies http://n.pr/171vPse


http://n.pr/171vPse
A DEET-Like Mosquito Spray That Smells Like Jasmine Or Grapes? http://n.pr/1c62zbK


http://n.pr/1c62zbK
Fish Guidelines For Pregnant Women May Be Too Strict, Study Suggests http://n.pr/1701tGu


http://n.pr/1701tGu
Federal Funds For Meals On Wheels Tied Up In Shutdown http://n.pr/16ZVdyJ


http://n.pr/16ZVdyJ
Is It Time To Cool It On Kale Already? http://n.pr/16ZDdV5


http://n.pr/16ZDdV5
Why Eye Contact Can Fail To Win People Over http://n.pr/1c5poMD


http://n.pr/1c5poMD
Health Care Act Reminds Young Adults They're Not Invincible http://n.pr/1c4ZbxV


http://n.pr/1c4ZbxV
Tech Problems Plague First Day Of Health Exchange Rollout http://n.pr/1c3iszu


http://n.pr/1c3iszu
Can Millet Take On Quinoa? First, It'

http://n.pr/17gb3ba
Health Exchange Outreach Targets Latinos http://n.pr/1abLJXG


http://n.pr/1abLJXG
Big Weight Loss For Diabetics, But No Drop In Heart Risk http://n.pr/14pvcZZ


http://n.pr/14pvcZZ
Will GMOs Help Protect Ugandan Families Against Hunger? http://n.pr/17bptJK


http://n.pr/17bptJK
Why Morning-After Pill Won't Stop All Unintended Pregnancies http://n.pr/17aSepZ


http://n.pr/17aSepZ
Top Medicare Prescribers Rake In Speaking Fees From Drugmakers http://n.pr/14nj2AH


http://n.pr/14nj2AH
Our Beef With BuzzFeed's Viral Article On 8 Dangerous Foods http://n.pr/19md2iA


http://n.pr/19md2iA
Could LeBron and RGIII Help Sell The Affordable Care Act? http://n.pr/130B6Ec


http://n.pr/130B6Ec
Doctors Say Wait Longer Before Treating Kids' Sinus Infections http://n.pr/176OLc7


http://n.pr/176OLc7
The Unease Over Classifying Obesity As A Disease http://n.pr/176uD9Q


http://n.pr/176uD9Q
Proposed Changes In Organ Donation Stir Debate http://n.pr/19jiPVU


http://n.pr/19jiPVU
Gloom

New Voices For The Voiceless: Synthetic Speech Gets An Upgrade http://n.pr/YbZ6iB


http://n.pr/YbZ6iB
Depression And Anxiety Could Be Fukushima's Lasting Legacy http://n.pr/YbZ3TY


http://n.pr/YbZ3TY
Aspirin Vs. Melanoma: Study Suggests Headache Pill Prevents Deadly Skin Cancer http://n.pr/YbZ3DC


http://n.pr/YbZ3DC
Career Suicide Or Lifesaver? Why A Professional Foodie Went Vegetarian http://n.pr/VT1Anm


http://n.pr/VT1Anm
Gluten Goodbye: One-Third Of Americans Say They're Trying to Shun It http://n.pr/14FxoM2


http://n.pr/14FxoM2
Flu Risk And Weather: It's Not The Heat, It's The Humidity http://n.pr/ZnxHM2


http://n.pr/ZnxHM2
Can the Anti-Aging Secret Be Found in...Red Wine? http://n.pr/10hNKK5


http://n.pr/10hNKK5
Could A 'Brain Pacemaker' Someday Treat Severe Anorexia? http://n.pr/10hNMS8


http://n.pr/10hNMS8
A Man's Journey From Nepal To Texas Triggers Global TB Scramble http://n.pr/YHt3Hn


http://n.pr/YHt3Hn
If Caffeine Can Boost The Memory Of Bees, Can It Help Us, Too? 


http://n.pr/ZInit6
Legalizing And Regulating Pot: A Growth Industry http://n.pr/Sl0lH8


http://n.pr/Sl0lH8
OK To Eat Before Rolling Up Sleeve For Cholesterol Test http://n.pr/ZE0tXu


http://n.pr/ZE0tXu
Death, But Softly http://n.pr/ZCjAkM


http://n.pr/ZCjAkM
High-Deductible Health Plans Can Cost Patients A Discount http://n.pr/X3e4Kr


http://n.pr/X3e4Kr
Missouri, Kansas Reject Health Exchanges http://n.pr/ZAqqY5


http://n.pr/ZAqqY5
Danes May Bring Back Butter As Government Rolls Back Fat Tax http://n.pr/X21bAd


http://n.pr/X21bAd
Health Insurance Exchanges Explained http://n.pr/ZA6NPV


http://n.pr/ZA6NPV
Malaria-Like Disease Follows Lyme's Path In New England http://n.pr/VYcE2n


http://n.pr/VYcE2n
Georgia Immigration Law Trips Up Doctors And Nurses http://n.pr/SgrvyY


http://n.pr/SgrvyY
How To Start Talking Details With Aging Parents http://n.pr/Tv5oXq


http://n.pr/Tv5oXq
Struggle For Smarts? How Eastern And Western Cultures Tackle Learning http://n.pr/WX1FHS


http://n.pr/W

,tweetid,date,title,url
0,585113044621066240,Mon Apr 06 16:13:15 +0000 2015,RT @JennyAGold: Maybe You Should Skip That Ann...,http://www.npr.org/blogs/health/2015/04/06/397...
1,547824165185536000,Wed Dec 24 18:40:33 +0000 2014,Would You Like Health Insurance With Those Sto...,http://n.pr/1CKulog
2,547763897638199296,Wed Dec 24 14:41:04 +0000 2014,New Blood Donation Rules Would Still Exclude M...,http://n.pr/1CJ9IsG
3,547671560471859200,Wed Dec 24 08:34:09 +0000 2014,Obama Administration Downplays Court Challenge...,http://n.pr/1CHaDd6
4,547671553609981952,Wed Dec 24 08:34:08 +0000 2014,"Christmas In Liberia: Ebola Fears, No Snow, Ho...",http://n.pr/1CHaAxQ


In [6]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@id="storytext"]/p/text()',
               '//div[@class="tags"]/ul/li/a/text()',
               '//div[@id="storybyline"]/div/div/div/p/a/text()'
               ]

In [7]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@id="storytext"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
            # Tags
            tags = xpath_selector.xpath('//div[@class="tags"]/ul/li/a/text()').extract()     
            print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@id="storytext"]/div/p/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': tags,'author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("==================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://www.npr.org/blogs/health/2015/04/06/397100748/maybe-you-should-skip-that-annual-physical
3703
7
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  3703
tags  :  7


scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://n.pr/1CKulog
2856
4
0
contents  :  2856
tags  :  4
Header:   200
http://n.pr/1CJ9IsG
6141
5
0
contents  :  6141
tags  :  5
Header:   200
http://n.pr/1CHaDd6
3526
5
0
contents  :  3526
tags  :  5
Header:   200
http://n.pr/1CHaAxQ
2593
3
0
contents  :  2593
tags  :  3
Header:   200
http://n.pr/1CHaCFX
4020
4
0
contents  :  4020
tags  :  4
Header:   200
http://n.pr/1CwBBH8
3275
5
0
contents  :  3275
tags  :  5
Header:   200
http://n.pr/1CvRzBi
2972
4
0
contents  :  2972
tags  :  4
Header:   200
http://n.pr/1CC0eiQ
1970
1
0
contents  :  1970
tags  :  1
author  :  0

Header:   200
http://n.pr/1CtPkOZ
3729
2
0
contents  :  3729
tags  :  2
Header:   200
http://n.pr/1Cr8sND
5514
2
0
contents  :  5514
tags  :  2
Header:   200
http://n.pr/1Cr4a8N
245
0
0
contents  :  245
tags  :  0
Header:   200
http://n.pr/1CqGvp1
2154
4
0
contents  :  2154
tags  :  4
Header:   200
http://n.pr/1CsVIDk
4552
4
0
contents  :  4552
tags  :  4
Header:   200
http://n.pr/1Cix2QD
169
1
0
contents  

Header:   200
http://n.pr/1CLBFnt
1869
1
0
contents  :  1869
tags  :  1
author  :  0

Header:   200
http://n.pr/1CLBFnj
4246
4
0
contents  :  4246
tags  :  4
Header:   200
http://n.pr/1CK58hy
4901
4
0
contents  :  4901
tags  :  4
Header:   200
http://n.pr/1Ao3fSR
2170
3
0
contents  :  2170
tags  :  3
Header:   200
http://n.pr/1CJUojo
182
0
0
contents  :  182
tags  :  0
Header:   200
http://n.pr/1CJUpDI
221
0
0
contents  :  221
tags  :  0
Header:   200
http://n.pr/1AmOhwo
6935
4
0
contents  :  6935
tags  :  4
Header:   200
http://n.pr/1AmBPNa
4659
4
0
contents  :  4659
tags  :  4
Header:   200
http://n.pr/1CJdNRh
1672
3
0
contents  :  1672
tags  :  3
Header:   200
http://n.pr/1CJaIkb
989
2
0
contents  :  989
tags  :  2
Header:   200
http://n.pr/1AjhBDX
4223
4
0
contents  :  4223
tags  :  4
Header:   200
http://n.pr/1yP5Oxd
3343
2
0
contents  :  3343
tags  :  2
Header:   200
http://n.pr/1CGjwr5
268
0
0
contents  :  268
tags  :  0
Header:   200
http://n.pr/1CGjuj7
4696
5
0
contents  :  46

Header:   200
http://n.pr/1qgcAM2
151
0
0
contents  :  151
tags  :  0
Header:   200
http://n.pr/1qgcAvy
3252
4
0
contents  :  3252
tags  :  4
Header:   200
http://n.pr/1qg5KpW
4562
3
0
contents  :  4562
tags  :  3
Header:   200
http://n.pr/11lnUdD
3799
4
0
contents  :  3799
tags  :  4
Header:   200
http://n.pr/1qf5qb0
4026
2
0
contents  :  4026
tags  :  2
Header:   200
http://n.pr/1qeXTJ4
3249
3
0
contents  :  3249
tags  :  3
Header:   200
http://n.pr/1qeXTc8
3450
4
0
contents  :  3450
tags  :  4
Header:   200
http://n.pr/11kmgZX
3034
2
0
contents  :  3034
tags  :  2
Header:   200
http://n.pr/11iWVPY
175
0
0
contents  :  175
tags  :  0
Header:   200
http://n.pr/1BHVEmr
3481
3
0
contents  :  3481
tags  :  3
Header:   200
http://n.pr/11cAQCE
209
0
0
contents  :  209
tags  :  0
Header:   200
http://n.pr/11ayMLo
4846
4
0
contents  :  4846
tags  :  4
Header:   200
http://n.pr/11a4sk4
2950
4
0
contents  :  2950
tags  :  4
Header:   200
http://n.pr/117D9H6
3332
4
0
contents  :  3332
tags  :  

...

Header:   200
http://n.pr/1fGsOXX
5052
5
0
contents  :  5052
tags  :  5
Header:   200
http://n.pr/1cqplI8
2650
0
0
contents  :  2650
tags  :  0
Header:   200
http://n.pr/1cpfT7T
206
0
0
contents  :  206
tags  :  0
Header:   200
http://n.pr/1fFiS0S
206
0
0
contents  :  206
tags  :  0
Header:   200
http://n.pr/1coEwS3
4338
3
0
contents  :  4338
tags  :  3
Header:   200
http://n.pr/1fF0khl
4095
1
0
contents  :  4095
tags  :  1
author  :  0

Header:   200
http://n.pr/1cn4HZs
2888
3
0
contents  :  2888
tags  :  3
Header:   200
http://n.pr/1cn05lP
3318
2
0
contents  :  3318
tags  :  2
Header:   200
http://n.pr/1cmKJ0Q
3318
2
0
contents  :  3318
tags  :  2
Header:   200
http://n.pr/1fDWLbi
295
0
0
contents  :  295
tags  :  0
Header:   200
http://n.pr/1fDWLbk
2400
2
0
contents  :  2400
tags  :  2
Header:   200
http://n.pr/1clAABD
4110
3
0
contents  :  4110
tags  :  3
Header:   200
http://n.pr/1fCZbqG
7654
2
0
contents  :  7654
tags  :  2
Header:   200
http://n.pr/1cl1dGH
7654
2
0
contents  :  

Header:   200
http://n.pr/1eL3z6m
4538
3
0
contents  :  4538
tags  :  3
Header:   200
http://n.pr/1bBk3Jq
6072
3
0
contents  :  6072
tags  :  3
['ERRORS'] 403 , at http://HealthCare.gov
Header:   200
http://n.pr/1eJd0mM
4040
4
0
contents  :  4040
tags  :  4
Header:   200
http://n.pr/1bBbAWE
142
0
0
contents  :  142
tags  :  0
Header:   200
http://n.pr/1eJ4hRK
3716
3
0
contents  :  3716
tags  :  3
Header:   200
http://n.pr/1eIVaAs
1666
4
0
contents  :  1666
tags  :  4
Header:   200
http://n.pr/1bABv0q
6432
4
0
contents  :  6432
tags  :  4
Header:   200
http://n.pr/1eI9B7T
360
0
0
contents  :  360
tags  :  0
Header:   200
http://n.pr/1bzLu6h
2958
3
0
contents  :  2958
tags  :  3
Header:   200
http://n.pr/1bykTGF
4984
3
0
contents  :  4984
tags  :  3
Header:   200
http://n.pr/1eGDgyd
4133
5
0
contents  :  4133
tags  :  5
Header:   200
http://n.pr/1bxXsNO
4984
3
0
contents  :  4984
tags  :  3
Header:   200
http://n.pr/1eFlX0l
2118
4
0
contents  :  2118
tags  :  4
Header:   200
http://n.pr/

Header:   200
http://n.pr/1dGLXbq
5789
0
0
contents  :  5789
tags  :  0
Header:   200
http://n.pr/Hs9QoH
1330
1
0
contents  :  1330
tags  :  1
author  :  0

Header:   200
http://n.pr/1gWdiIF
2606
5
0
contents  :  2606
tags  :  5
Header:   200
http://n.pr/1gW7JtW
947
2
0
contents  :  947
tags  :  2
Header:   200
http://n.pr/HrMbVu
3482
4
0
contents  :  3482
tags  :  4
Header:   200
http://n.pr/HrK4RF
2454
2
0
contents  :  2454
tags  :  2
Header:   200
http://n.pr/1gVTghE
931
4
0
contents  :  931
tags  :  4
Header:   200
http://n.pr/HrBbYb
10596
2
0
contents  :  10596
tags  :  2
Header:   200
http://n.pr/Hr87Qw
10596
2
0
contents  :  10596
tags  :  2
Header:   200
http://n.pr/1gUrhPd
4669
3
0
contents  :  4669
tags  :  3
Header:   200
http://n.pr/1gUpsC2
4235
6
0
contents  :  4235
tags  :  6
Header:   200
http://n.pr/1gTnKRf
343
0
0
contents  :  343
tags  :  0
Header:   200
http://n.pr/1gSWBxZ
931
4
0
contents  :  931
tags  :  4
Header:   200
http://n.pr/1gSndiB
660
1
0
contents  :  660


Header:   200
http://n.pr/GWDK58
4614
2
0
contents  :  4614
tags  :  2
Header:   200
http://n.pr/GW1jeD
7195
0
0
contents  :  7195
tags  :  0
Header:   200
http://n.pr/GV85RI
211
0
0
contents  :  211
tags  :  0
Header:   200
http://n.pr/1ah220C
5040
5
0
contents  :  5040
tags  :  5
Header:   200
http://n.pr/GTb8dc
3531
5
0
contents  :  3531
tags  :  5
Header:   200
http://n.pr/GSAqIv
2936
6
0
contents  :  2936
tags  :  6
Header:   200
http://n.pr/GSpJ8F
3108
4
0
contents  :  3108
tags  :  4
Header:   200
http://n.pr/1ag3bFS
3282
0
0
contents  :  3282
tags  :  0
Header:   200
http://n.pr/1ag3bW8
1425
0
0
contents  :  1425
tags  :  0
Header:   200
http://n.pr/GShfhQ
4117
0
0
contents  :  4117
tags  :  0
Header:   200
http://n.pr/1ag3bWa
16587
0
0
contents  :  16587
tags  :  0
Header:   200
http://n.pr/GShfhU
4645
0
0
contents  :  4645
tags  :  0
Header:   200
http://n.pr/GShd9R
4804
0
0
contents  :  4804
tags  :  0
Header:   200
http://n.pr/GSeBsq
252
0
0
contents  :  252
tags  :  0
Head

Header:   200
http://n.pr/1bM9KFL
1505
0
0
contents  :  1505
tags  :  0
Header:   200
http://n.pr/16PkL2h
126
0
0
contents  :  126
tags  :  0
Header:   200
http://n.pr/16PkIU9
202
0
0
contents  :  202
tags  :  0
Header:   200
http://n.pr/16PhrEt
4039
3
0
contents  :  4039
tags  :  3
Header:   200
http://n.pr/16NN9lv
3737
4
0
contents  :  3737
tags  :  4
Header:   200
http://n.pr/1at3UW7
2599
5
0
contents  :  2599
tags  :  5
Header:   200
http://n.pr/1fK5ZRg
1942
3
0
contents  :  1942
tags  :  3
Header:   200
http://n.pr/1asT7LB
5976
3
0
contents  :  5976
tags  :  3
Header:   200
http://n.pr/1fJEirU
1423
3
0
contents  :  1423
tags  :  3
Header:   200
http://n.pr/1asuFKf
5921
0
0
contents  :  5921
tags  :  0
Header:   200
http://n.pr/1fJobux
4503
3
0
contents  :  4503
tags  :  3
Header:   200
http://n.pr/1asoPZh
4503
3
0
contents  :  4503
tags  :  3
Header:   200
http://n.pr/1as7LTn
3123
1
0
contents  :  3123
tags  :  1
author  :  0

Header:   200
http://n.pr/1arYPNT
5428
4
0
contents  :

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape

In [ ]:
frame = pd.concat([frame,df_temp],axis=1)
df_temp.head()

In [ ]:
frame.to_csv('./run_data/npr_scrapt_v2.csv', sep='|', encoding='utf-8', index=False)